In [5]:
using GuSTO
# include("../src/GuSTO.jl")
using AstrobeeRobot
using GeometryTypes: Point3, HyperSphere
using MeshCat, MeshCatMechanisms, CoordinateTransformations
import ColorTypes: RGBA

In [6]:
robot = Astrobee3D()
model = AstrobeeSE3()
env = ISSCorner();

x_init = [11.2; -0.8; 5.6]
x_goal = [10.2; 6.9; 4.2]

# x_init = [x_init; zeros(3); quat2mrp(sqrt(1/3)*[1.; 0.; 1.; 1.]); zeros(3)]
# x_goal = [x_goal; zeros(3); quat2mrp([-0.5; 0.5; -0.5; 0.5]); zeros(3)]
x_init = [x_init; zeros(3); GuSTO.quat2mrp([0.; 0.; 0.; 1.]); zeros(3)]
x_goal = [x_goal; zeros(3); GuSTO.quat2mrp([0.; 0.; 0.; -1.]); zeros(3)]
# x_init = [x_init; zeros(3); quat2mrp([0.; 0.; 0.; 1.]); zeros(3)]
# x_goal = [x_goal; zeros(3); quat2mrp([0.; 0.; 0.; -1.]); zeros(3)]

N = 30
tf_guess = 70.

PD = ProblemDefinition(robot, model, env, x_init, x_goal);
TOPgusto = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSgusto = TrajectoryOptimizationSolution(TOPgusto)
solve_SCP!(TOSgusto, TOPgusto, solve_gusto_jump!, init_traj_straightline, "Gurobi", OutputFlag=0)

┌ Warning: `Array{T, 1}(m::Int) where T` is deprecated, use `Array{T, 1}(undef, m)` instead.
│   caller = ISSCorner{Float64}() at iss_corner.jl:12
└ @ Main /home/bylard/.julia/dev/GuSTO/src/environment/iss_corner.jl:12
┌ Warning: `Pkg.dir(pkgname, paths...)` is deprecated; instead, do `import GuSTO; joinpath(dirname(pathof(GuSTO)), "..", paths...)`.
└ @ Pkg.API /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v0.7/Pkg/src/API.jl:454
┌ Warning: `Array{T, 1}(m::Int) where T` is deprecated, use `Array{T, 1}(undef, m)` instead.
│   caller = ISSCorner{Float64}() at iss_corner.jl:19
└ @ Main /home/bylard/.julia/dev/GuSTO/src/environment/iss_corner.jl:19
┌ Warning: `Array{T, 1}(m::Int) where T` is deprecated, use `Array{T, 1}(undef, m)` instead.
│   caller = ISSCorner{Float64}() at iss_corner.jl:25
└ @ Main /home/bylard/.julia/dev/GuSTO/src/environment/iss_corner.jl:25
┌ Warning: `x ^ p::Integer` is deprecated, use `Base.power_by_squaring(x, p)` instead.
│   caller = macro exp

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


┌ Warning: Unable to use cached solution to warmstart problem.
│                 (Perhaps the number of variables or constraints in the problem have changed since you last solved it?)
│                 Warmstart may be ineffective.
└ @ Convex /home/bylard/.julia/packages/Convex/Y3Bcx/src/solution.jl:66
┌ Warning: Unable to use cached solution to warmstart problem.
│                 (Perhaps the number of variables or constraints in the problem have changed since you last solved it?)
│                 Warmstart may be ineffective.
└ @ Convex /home/bylard/.julia/packages/Convex/Y3Bcx/src/solution.jl:66


In [7]:
@show TOSgusto.SCPS.converged
@show TOSgusto.SCPS.iterations
@show TOSgusto.SCPS.total_time
@show TOSgusto.SCPS.accept_solution
@show TOSgusto.SCPS.prob_status
@show TOSgusto.SCPS.convergence_measure
@show TOSgusto.SCPS.param.alg.ω_vec
@show TOSgusto.SCPS.param.alg.Δ_vec 
;

(TOSgusto.SCPS).converged = true
(TOSgusto.SCPS).iterations = 3
(TOSgusto.SCPS).total_time = 14.278333732
(TOSgusto.SCPS).accept_solution = Bool[true, true, true, true]
(TOSgusto.SCPS).prob_status = Symbol[:NA, :Optimal, :Optimal, :Optimal]
(TOSgusto.SCPS).convergence_measure = [0.0, 0.0649572, 0.0120937, 0.0020919]
(((TOSgusto.SCPS).param).alg).ω_vec = [1.0, 1.0, 1.0, 1.0]
(((TOSgusto.SCPS).param).alg).Δ_vec = [10.0, 10.0, 10.0, 10.0]


In [9]:
# Animate Astrobee trajectory
vis = Visualizer()
delete!(vis)

vis[:goal]
for (idx,obs) in enumerate(env.keepout_zones)
    setobject!(vis[:goal][:goal], 
        Object(HyperSphere(Point3(x_goal[1:3]), 0.1),
            MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))
end

vis[:workspace]
for (idx,ws) in enumerate(env.keepin_zones)
    if idx in (5,8)
        setobject!(vis[:workspace][Symbol(string("ws",idx))],
            Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3), depthWrite=false)))
    else
        setobject!(vis[:workspace][Symbol(string("ws",idx))],
            Object(ws, MeshBasicMaterial(color=RGBA(0.95,0.93,0.26,0.3))))
    end
end

vis[:obstacle]
for (idx,ws) in enumerate(env.obstacle_set)
    setobject!(vis[:workspace][Symbol(string("ws",idx+length(env.keepin_zones)))],
        Object(ws,MeshBasicMaterial(color=RGBA(0.95,0.26,0.26,0.3))))
end

ab = Astrobee()
mvis = MechanismVisualizer(
    ab.mechanism,
    URDFVisuals(AstrobeeRobot.urdfpath(), package_path=[dirname(dirname(AstrobeeRobot.urdfpath()))]),
    vis);

speed_factor = 1

Qs = Vector{Vector{Float64}}(0)
for k in 1:speed_factor:N
    q = [GuSTO.quat_inv(GuSTO.mrp2quat(TOSgusto.SCPS.traj.X[7:9,k]));TOSgusto.SCPS.traj.X[1:3,k]]
#     q = [quat_inv(mrp2quat(TOSgusto.SCPS.traj.X[7:9,k]));TOSgusto.SCPS.traj.X[1:3,k]]
    push!(Qs,q)
end

trans = Translation(14., -1., 7.)
rot = LinearMap(RotZ(-0.6)) ∘ LinearMap(RotY(-0.2))
settransform!(vis["/Cameras/default"], trans ∘ rot)
setprop!(vis["/Cameras/default/rotated/<object>"], "zoom", 1.9)
setprop!(vis["/Cameras/default/rotated/<object>"], "near", 0.05)

sleep(3)
setanimation!(mvis,1:length(Qs),Qs)

plot_in_cell = false
plot_in_cell ? IJuliaCell(vis) : open(vis)

instantiated a floating joint


┌ Warning: `Array{T, 1}(m::Int) where T` is deprecated, use `Array{T, 1}(undef, m)` instead.
│   caller = top-level scope at In[9]:36
└ @ Core In[9]:36
┌ Info: Serving MeshCat visualizer at http://127.0.0.1:8700
└ @ MeshCat /home/bylard/.julia/packages/MeshCat/jt9Xz/src/servers.jl:7
┌ Info: Listening on: Sockets.InetAddr{Sockets.IPv4}(ip"127.0.0.1", 0x21fc)
└ @ HTTP.Servers /home/bylard/.julia/packages/HTTP/YjRCz/src/Servers.jl:301
/usr/bin/google-chrome-stable: /home/bylard/.julia/v0.6/Conda/deps/usr/lib/libuuid.so.1: no version information available (required by /usr/bin/google-chrome-stable)
/opt/google/chrome/chrome: /home/bylard/.julia/v0.6/Conda/deps/usr/lib/libuuid.so.1: no version information available (required by /opt/google/chrome/chrome)
GLib-GIO-Message: 18:02:54.159: Using the 'memory' GSettings backend.  Your settings will not be saved or shared with other applications.
[12068:12096:0107/180254.192255:ERROR:bus.cc(396)] Failed to connect to the bus: Failed to connect to 

Process(`xdg-open http://127.0.0.1:8700`, ProcessExited(0))

Opening in existing browser session.


/proc/self/exe: /home/bylard/.julia/v0.6/Conda/deps/usr/lib/libuuid.so.1: no version information available (required by /proc/self/exe)
┌ Info: Accept (0):  🔗    0↑     0↓    1s 127.0.0.1:8700:8700 ≣16
└ @ HTTP.Servers /home/bylard/.julia/packages/HTTP/YjRCz/src/Servers.jl:343
┌ Info: Accept (1):  🔗    0↑     0↓    1s 127.0.0.1:8700:8700 ≣16
└ @ HTTP.Servers /home/bylard/.julia/packages/HTTP/YjRCz/src/Servers.jl:343
┌ Info: Accept (2):  🔗    0↑     0↓    0s 127.0.0.1:8700:8700 ≣16
└ @ HTTP.Servers /home/bylard/.julia/packages/HTTP/YjRCz/src/Servers.jl:343
┌ Info: Accept (3):  🔗    0↑     0↓    0s 127.0.0.1:8700:8700 ≣16
└ @ HTTP.Servers /home/bylard/.julia/packages/HTTP/YjRCz/src/Servers.jl:343
┌ Warning: `wait(t::Task)` is deprecated, use `fetch(t)` instead.
│   caller = macro expansion at task.jl:265 [inlined]
└ @ Core ./task.jl:265
Error handling websocket connection:
WebSockets.WebSocketClosedError("ws|server respond to OPCODE_CLOSE 1001:Going Away")┌ Info: Closed (3):  💀    1↑    